In [1]:
import re, datetime, sys, os
import imageio
import json
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import requests

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from operator import itemgetter
from pathlib import Path
from textblob import TextBlob, Word
from textblob.sentiments import NaiveBayesAnalyzer
from wordcloud import WordCloud
import pprint

In [2]:

result = sp.search("asdfasdfa", limit=1, type='track')


NameError: name 'sp' is not defined

In [ ]:

try:
    result = sp.search("Planet P Project Pink World", limit=1, type='track')
    a = result['tracks']['items'][0]
    print(a)
except:
    print("UNKNOWN")
    

In [39]:
result = sp.search("Temptations - I'm Losing You", limit=1, type='track')
a = result['tracks']['items'][0]
spotify_id = a['id']

In [40]:
spotify_id

'6y8o4V2R2dSt2ijgL8C72W'

In [3]:
# From Sam
def get_track_audio_feautures(track_id):
    track_results = sp.audio_features(track_id)
    danceability=track_results[0]['danceability']
    energy = track_results[0]['energy']
    key = (track_results[0]['key'])
    loudness = track_results[0]['loudness']
    mode = track_results[0]['mode']
    speechiness = track_results[0]['speechiness']
    acousticness= track_results[0]['acousticness']
    instrumentalness = track_results[0]['instrumentalness']
    liveness = track_results[0]['liveness']
    valence = track_results[0]['valence']
    tempo = track_results[0]['tempo']
    track_href = track_results[0]['track_href']
    analysis_url = track_results[0]['analysis_url']
    time_signature = track_results[0]['time_signature']
    table = pd.DataFrame({'Danceability':danceability,
                          'Energy':energy,
                          'Key':key,
                          'Loudness':loudness,
                          'Mode': mode,
                          'Speechiness':speechiness,
                          'Acousticness': acousticness,
                          'Instrumentalness':instrumentalness,
                          'Liveness': liveness,
                          'Valence':valence,
                          'Tempo':tempo,
                          'Time Signature':time_signature,
                          'Track Link':track_href,
                          'Spotify Analysis':analysis_url
                         },
                          index=[0])
    return(table)

In [ ]:
test = get_track_audio_feautures('6RX5iL93VZ5fKmyvNXvF1r')

In [ ]:
test

In [ ]:
test = sp.artist('6RX5iL93VZ5fKmyvNXvF1r')

### Set up Spotify Credentials and functions

In [4]:
SPOTIPY_CLIENT_ID ='d0e9cdee463948d8985ec7d7c5ce0aac'
SPOTIPY_CLIENT_SECRET ='d864675d0b5d407f913d94a50c6cf7a5'
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
def get_spotify_id(search):
    try:
        result = sp.search(search, limit=1, type='track')
        a = result['tracks']['items'][0]
        spotify_id = a['id']
        return spotify_id
    except: 
        return None

In [7]:
get_spotify_id('Beyonce run the world')

'1uXbwHHfgsXcUKfSZw5ZJ0'

### Load CAL10K Data Set for Training

In [1]:
cal10k = pd.read_csv('analysis_data_sources/CAL10K_genre_noAudioFeatures.txt', sep='\t',header=None,names=['Song', 'Genre'])

NameError: name 'pd' is not defined

In [69]:
cal10k.head()

,Song,Genre,Spotify_ID,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Luther Vandross - If Only For One Night,adult contemporary,2p4WnpR3H16ywshkL2Fp1n,0.498000,0.249000,9.000000,-13.619000,0.000000,0.035500,0.755000,0.000009,0.092700,0.193000,110.636000,256240.000000,4.000000
1,Duran Duran - Come Undone,adult contemporary,0yfNXxlyXdmP0ue1iJijx1,0.542000,0.793000,8.000000,-8.203000,1.000000,0.053000,0.042300,0.000004,0.081400,0.708000,173.931000,256520.000000,4.000000
2,Tears For Fears - Shout,adult contemporary,2gQaQUhDCNGfBVXTvxAmXQ,0.511000,0.942000,0.000000,-10.407000,1.000000,0.042900,0.174000,0.000019,0.139000,0.498000,97.909000,393667.000000,4.000000
3,Marvin Gaye - Sexual healing,adult contemporary,3VZmChrnVW8JK6ano4gSED,0.805000,0.601000,0.000000,-9.700000,0.000000,0.057900,0.119000,0.000129,0.058100,0.746000,94.382000,238600.000000,4.000000
4,Jack Johnson - Better Together,adult contemporary,4VywXu6umkIQ2OS0m1I79y,0.868000,0.347000,5.000000,-9.529000,1.000000,0.053600,0.308000,0.000020,0.110000,0.659000,110.059000,207680.000000,4.000000


In [61]:
# cal10k['Spotify_ID'] = get_spotify_id(cal10k['Song'])
cal10k['Spotify_ID'] = cal10k['Song'].apply(get_spotify_id)

In [68]:
cal10k.to_csv('analysis_data_sources/CAL10K_with_Spotify_WithFeatures.tab', sep='\t')

### Get Features from Spotify

In [65]:
def get_features(spotify_id):
    try:
        feature=sp.audio_features(spotify_id)[0]
        return format(feature['danceability'], 'f' ) \
                ,format(feature['energy'], 'f' ) \
                ,format(feature['key'], 'f' ) \
                ,format(feature['loudness'], 'f' ) \
                ,format(feature['mode'], 'f' ) \
                ,format(feature['speechiness'], 'f' ) \
                ,format(feature['acousticness'], 'f' ) \
                ,format(feature['instrumentalness'], 'f' ) \
                ,format(feature['liveness'], 'f' ) \
                ,format(feature['valence'], 'f' ) \
                ,format(feature['tempo'], 'f' ) \
                ,format(feature['duration_ms'], 'f' ) \
                ,format(feature['time_signature'], 'f' )
    except:
        return (0,) * 13 #Return 13 zeros

In [67]:
cal10k['danceability']    \
,cal10k['energy']         \
,cal10k['key']            \
,cal10k['loudness']       \
,cal10k['mode']           \
,cal10k['speechiness']    \
,cal10k['acousticness']   \
,cal10k['instrumentalness'] \
,cal10k['liveness']       \
,cal10k['valence']        \
,cal10k['tempo']          \
,cal10k['duration_ms']    \
,cal10k['time_signature'] = zip(*cal10k['Spotify_ID'].apply(get_features))

In [9]:
def get_features(spotify_id):
    try:
        feature=sp.audio_features(spotify_id)[0]
        return format(feature['danceability'], 'f' ) \
                ,format(feature['energy'], 'f' ) \
                ,format(feature['key'], 'f' ) \
                ,format(feature['loudness'], 'f' ) \
                ,format(feature['mode'], 'f' ) \
                ,format(feature['speechiness'], 'f' ) \
                ,format(feature['acousticness'], 'f' ) \
                ,format(feature['instrumentalness'], 'f' ) \
                ,format(feature['liveness'], 'f' ) \
                ,format(feature['valence'], 'f' ) \
                ,format(feature['tempo'], 'f' ) \
                ,format(feature['duration_ms'], 'f' ) \
                ,format(feature['time_signature'], 'f' )
    except:
        return (0,) * 13

In [10]:
print(get_features('1uXbwHHfgsXcUKfSZw5ZJ0'))

('0.733000', '0.899000', '0.000000', '-4.237000', '1.000000', '0.143000', '0.004960', '0.000047', '0.372000', '0.760000', '127.086000', '236093.000000', '4.000000')
